## Random forest algorithm with pipeline concept 

In [1]:
import seaborn as sns
df = sns.load_dataset('tips')

In [2]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
## We want to predict the dinner time from this dataset 

In [4]:
# from the data we can see that there are values with the categorical features, to convert them into the numerical
# we will use the encoding 
from sklearn.preprocessing import LabelEncoder

In [6]:
encoder = LabelEncoder()
df['time'] = encoder.fit_transform(df['time'])

In [10]:
df['time'].value_counts()  # 0 - dinner and 1 = lunch 

0    176
1     68
Name: time, dtype: int64

In [11]:
y = df['time']
df.drop(columns='time', axis =1)

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [12]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int32

In [13]:
X = df

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer ## HAndle Missing Values
from sklearn.preprocessing import StandardScaler ## Feature Scaling
from sklearn.preprocessing import OneHotEncoder ## categorical to numerical
from sklearn.compose import ColumnTransformer

In [16]:
categorical_cols = ['sex', 'smoker','day']
numerical_cols = ['total_bill', 'tip','size']

In [17]:
# feature engineering automation 
# Numerical pipeline 

num_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')), # for handling missing values 
        ('scaler',StandardScaler() ) # feature scaling 
    ]
)


cat_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')), # handling missing values 
        ('encoder', OneHotEncoder()) # categorical features to numericals 
    ]
)

In [18]:
preprocessor = ColumnTransformer(
    [
        ('num_pipeline',num_pipeline, numerical_cols),
        ('cat_pipeline', cat_pipeline, categorical_cols) 
    ]
)

In [19]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [21]:
# model training automation 

models = {
    'Random forest': RandomForestClassifier(),
    'Logistic Regression': LogisticRegression(), 
    'Desicion Tree' : DecisionTreeClassifier()
}

In [22]:
from sklearn.metrics import accuracy_score

In [23]:
# making a function for an automatic model training 
def evaluate_model(X_train,y_train,X_test,y_test,models):
    

    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]

        # train model 
        model.fit(X_train,y_train)



        # Predict Testing data
        y_test_pred =model.predict(X_test)

        # Get accuracy for test data prediction

        test_model_score = accuracy_score(y_test,y_test_pred)

        report[list(models.keys())[i]] =  test_model_score



    return report

In [25]:
evaluate_model(X_train, y_train, X_test, y_test, models)

{'Random forest': 0.9591836734693877,
 'Logistic Regression': 1.0,
 'Desicion Tree': 0.9387755102040817}